## Install necessary dependencies

In [1]:

%pip install pyaudio SpeechRecognition keyboard spacy

import pyaudio
import keyboard

%pip install ffmpeg

%pip install pydub SpeechRecognition word2number

%pip install spacy

# Download spaCy language model
!python -m spacy download en_core_web_sm

# Import necessary modules
import re
from word2number import w2n
import spacy  # Replacing nltk with spacy
import json
from IPython.display import display, Javascript
import base64
import speech_recognition as sr
from pydub import AudioSegment
import io

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")



     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.2 MB 1.6 MB/s eta 0:00:08
      --------------------------------------- 0.2/12.2 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.4/12.2 MB 2.1 MB/s eta 0:00:06
     - -------------------------------------- 0.5/12.2 MB 2.3 MB/s eta 0:00:05
     -- ------------------------------------- 0.9/12.2 MB 3.3 MB/s eta 0:00:04
     ----- ---------------------------------- 1.6/12.2 MB 5.1 MB/s eta 0:00:03
     -------- ------------------------------- 2.7/12.2 MB 7.8 MB/s eta 0:00:02
     -------------- ------------------------- 4.3/12.2 MB 11.0 MB/s eta 0:00:01
     ------------------- -------------------- 6.1/12.2 MB 13.8 MB/s eta 0:00:01
     ------------------------ --------------- 7.3/12.2 MB 15.1 MB/s eta 0:00:01
     ---------------------------- ----------- 8.6/12.2 MB 16.1


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.2/12.8 MB 4.5 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/12.8 MB 12.0 MB/s eta 0:00:01
     -------- ------------------------------- 2.8/12.8 MB 19.9 MB/s eta 0:00:01
     ------------- -------------------------- 4.3/12.8 MB 22.9 MB/s eta 0:00:01
     ------------------- -------------------- 6.2/12.8 MB 26.4 MB/s eta 0:00:01
     ------------------------ --------------- 8.0/12.8 MB 28.3 MB/s eta 0:00:01
     ----------------------------- ---------- 9.3/12.8 MB 29.7 MB/s eta 0:00:01
     -------------------------------- ------ 10.6/12.8 MB 34.4 MB/s eta 0:00:01
     ----------------------------------- --- 11.8/12.8 MB 34.4 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 32.8 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 29.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\dxl33\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~pacy\\attrs.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\dxl33\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

## Define Recorder Class

In [ ]:

class Recorder:
    def __init__(self, silence_timeout=15):
        self.recognizer = sr.Recognizer()
        self.silence_timeout = silence_timeout

    def record_audio(self):
        with sr.Microphone() as source:
            print("Adjusting for ambient noise, please wait...")
            self.recognizer.adjust_for_ambient_noise(source)
            print("Recording... Speak now.")
            
            audio_data = self.recognizer.listen(source, timeout=self.silence_timeout, phrase_time_limit=self.silence_timeout)
            print("Recording stopped.")
            
            return audio_data


## Define Transcriber Class

In [ ]:

class Transcriber:
    def __init__(self):
        self.recognizer = sr.Recognizer()
    
    def transcribe_audio(self, audio_data):
        try:
            print("Transcribing...")
            prompt = self.recognizer.recognize_google(audio_data)
            print("You said: " + prompt)
            return prompt
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return ""


In [ ]:

## Record Audio Prompt (No Change)
def main():
    recorder = Recorder(silence_timeout=15)
    transcriber = Transcriber()

    audio_data = recorder.record_audio()
    prompt = transcriber.transcribe_audio(audio_data)
    
    if not prompt:
        print("No transcription available.")
    else:
        print(f"Transcribed Prompt: {prompt}")

    return prompt

prompt = main()


In [ ]:

## Type Prompt (No Change)
#Option to Type Prompt Instead
prompt = "I need to update tables 5 11 and 17 to have 3.25 foot sides and a height of 3 feet and use a leg insert of 1.25 inches make it 0.75 inches thick with corner fillets of 5.25 inches."


In [ ]:

## Find/Assign Variables (Updated to Use spaCy)
# Define keywords for actions
action_keywords = {
    "Create": ["create", "make", "build", "design", "generate"],
    "Modify": ["modify", "change", "adjust", "alter", "update"]
}

# Function to detect action using spaCy
def detect_action(text):
    # Use spaCy for tokenization
    doc = nlp(text.lower())
    
    # Initialize action variable
    detected_action = "UNKNOWN"
    
    # Check for CREATE keywords
    create_keywords = action_keywords["Create"]
    if any(token.text in create_keywords for token in doc):
        detected_action = "Create"

    # Check for MODIFY keywords
    modify_keywords = action_keywords["Modify"]
    if any(token.text in modify_keywords for token in doc):
        detected_action = "Modify"

    return detected_action

# Use the prompt to determine action
action = detect_action(prompt)
print(f"Detected Action: {action}")


In [ ]:

### Variables Extraction (Updated for spaCy Tokenization)
# Define possible synonyms for each parameter
parameter_synonyms = {
    "Height": ["height", "tall", "elevation", "high"],
    "Corner Fillet": ["corner fillet", "fillet", "rounded corner", "corner fillets", "fillets"],
    "Leg Insert": ["leg insert", "leg distance", "leg spacing", "inserts", "leg insert distance of "],
    "Sides": ["sides", "edges"],
    "Thickness": ["thickness", "thick", "depth"]
}

# Function to find the closest number to a keyword
def find_closest_value(keyword, text):
    # Tokenize text using spaCy
    doc = nlp(text)
    pattern = re.compile(r'(\b\d+(\.\d+)?\b)\s*(feet|foot|inches|inch|ft|in)?')
    matches = pattern.finditer(text)
    keyword_position = text.lower().find(keyword.lower())

    closest_distance = float('inf')
    closest_value = None

    for match in matches:
        number, _, unit = match.groups()
        start, end = match.span()

        # Calculate the distance between the keyword and the number
        distance = abs(keyword_position - start)

        if distance < closest_distance:
            closest_distance = distance
            closest_value = f"{number} {unit}".strip()

    return closest_value

# Extract parameters
parameters = {}

for parameter, synonyms in parameter_synonyms.items():
    for synonym in synonyms:
        if synonym in prompt:
            value = find_closest_value(synonym, prompt)
            if value:
                parameters[parameter] = value
                break

print(json.dumps(parameters, indent=2))


In [ ]:

### ID Extraction (No Change)
def extract_ids(text):
    text = text.lower()
    id_keywords = ['table', 'tables']
    ids = []

    words = text.split()

    for i, word in enumerate(words):
        if word in id_keywords:
            j = i + 1
            while j < len(words):
                if words[j].isdigit():
                    ids.append(int(words[j]))
                elif 'and' in words[j]:
                    j += 1
                    if j < len(words) and words[j].isdigit():
                        ids.append(int(words[j]))
                elif ',' in words[j]:
                    comma_separated_ids = [int(num) for num in words[j].split(',') if num.isdigit()]
                    ids.extend(comma_separated_ids)
                else:
                    break
                j += 1

    ids = sorted(set(ids))
    return ids

ids = extract_ids(prompt)
print(f"Modified IDs: {ids}")


In [ ]:

### Export Revit Parameters (No Change)
def extract_number(value):
    match = re.search(r'\d+(\.\d+)?', value)
    if match:
        return match.group()
    return "Not specified"

def prepare_revit_parameters(parameters, action, ids):
    revit_parameters = {
        "Action": action,
        "Table IDs": ids,
        "Parameters": {
            "Height": {
                "ParameterName": "Height",
                "Value": extract_number(parameters.get("Height", "Not specified")),
                "Unit": "feet"
            },
            "Thickness": {
                "ParameterName": "Thickness",
                "Value": extract_number(parameters.get("Thickness", "Not specified")),
                "Unit": "inches"
            },
            "Sides": {
                "ParameterName": "Number of Sides",
                "Value": extract_number(parameters.get("Sides", "Not specified")),
                "Unit": "feet"
            },
            "Leg Insert": {
                "ParameterName": "Leg Insert",
                "Value": extract_number(parameters.get("Leg Insert", "Not specified")),
                "Unit": "inches"
            },
            "Corner Fillet": {
                "ParameterName": "Corner Fillet",
                "Value": extract_number(parameters.get("Corner Fillet", "Not specified")),
                "Unit": "inches"
            }
        }
    }
    return revit_parameters

revit_parameters = prepare_revit_parameters(parameters, action, ids)
print(json.dumps(revit_parameters, indent=4))

json_filename = "Test2Parameter.json"

with open(json_filename, 'w') as json_file:
    json.dump(revit_parameters, json_file, indent=4)
